In [32]:
# Cell 1: Setup - minimal imports and notes
# Purpose: keep imports minimal for exploration and analysis

import os
import json
from pathlib import Path
import numpy as np
from PIL import Image

# Note: `transformers`, `torch`, and plotting libs are optional for later steps
# TODO: install required packages if needed: `pip install transformers torch matplotlib`


In [33]:
# Cell 2: Download dataset (default kagglehub cache)
# TODO: ensure you have kaggle credentials in ~/.kaggle/kaggle.json

try:
    import kagglehub
except Exception as e:
    print('kagglehub not installed; please install: pip install kagglehub')

# Download dataset (will use default cache: ~/.cache/kagglehub)
# NOTE: this cell intentionally performs download to default cache
# to keep reproducible behavior across environments

def download_dataset(dataset_id: str = "nghigia/hcmc-aqi-traffic-camera-dataset") -> str:
    """Download dataset via kagglehub.dataset_download and return path.

    Returns:
        path (str): local path to downloaded dataset (version folder)
    """
    # TODO: implement actual download call
    raise NotImplementedError("TODO: call kagglehub.dataset_download and return path")


In [34]:
# Cell 3: Verify files & basic metadata
# Purpose: list top-level folders, counts, and sample filenames

def summarize_data_folder(path: str, max_folders: int = 20) -> dict:
    """Return a summary dict with folder->num_images and sample files.

    TODOs:
    - list folders in path
    - for each folder count images and gather a few sample names
    - return a dict
    """
    # TODO: implement summarization
    raise NotImplementedError("TODO: implement summarize_data_folder")


# Example usage (learner):
# dataset_path = download_dataset()
# summary = summarize_data_folder(dataset_path)
# print(summary)


# Answer only these questions (don’t model yet):

## 1️⃣ What does each folder represent?

Is it a single camera?

A location?

A time span?

Check:

Number of images per folder

Timestamps (from filenames or JSON)

Resolution consistency


In [35]:
# Cell 4: Analysis skeleton - answer the core questions
# Purpose: Provide function stubs to determine camera/ location/ timespan


def count_images_per_folder(folder_path: str) -> dict:
    """Return mapping folder->image count.

    Expected: {folder_name: count}
    TODO: implement using os.listdir
    """
    raise NotImplementedError("TODO: implement count_images_per_folder")


def extract_timestamps_from_filenames(folder_path: str, pattern: str = None) -> list:
    """Attempt to parse timestamps from filenames.

    Hints:
    - filenames may contain UNIX timestamps or formatted strings
    - consider regex search for 4+ digit year or long epoch numbers
    """
    raise NotImplementedError("TODO: implement extract_timestamps_from_filenames")


def check_resolution_consistency(folder_path: str, sample_n: int = 5) -> dict:
    """Sample images and return resolutions seen per folder.

    Returns: {folder_name: set((w,h), ...)}
    """
    raise NotImplementedError("TODO: implement check_resolution_consistency")

# Example flow (learner):
# counts = count_images_per_folder(dataset_path)
# timestamps = extract_timestamps_from_filenames(some_folder)
# resolutions = check_resolution_consistency(some_folder)


In [36]:
# Cell 5: Utility - identify corrupted small PNGs (3KB or smaller)
# Purpose: Provide reusable function; removal left to user

def find_small_pngs(root_path: str, size_threshold_bytes: int = 3072) -> list:
    """Return list of filepaths for PNGs <= threshold.

    Learner action: call this, inspect results, and optionally remove files.
    """
    small_files = []
    for folder_name in os.listdir(root_path):
        folder_path = os.path.join(root_path, folder_name)
        if not os.path.isdir(folder_path):
            continue
        for fname in os.listdir(folder_path):
            if fname.lower().endswith('.png'):
                fp = os.path.join(folder_path, fname)
                try:
                    if os.path.getsize(fp) <= size_threshold_bytes:
                        small_files.append(fp)
                except Exception:
                    pass
    return small_files

# Example usage:
# smalls = find_small_pngs(dataset_path)
# print(len(smalls))
# To remove (careful):
# for f in smalls:
#     os.remove(f)


PNG that are unusable for modeling (e.g., corrupted files) are 3KB or smaller. These should be removed from the dataset before modeling.


## 2️⃣ Are there obvious failure frames?

Manually inspect:

Black frames

Extreme blur

Obstruction

Overexposure

Frozen scenes

In [37]:
# Cell 6: Frozen frame detection (exact duplicates)
# Purpose: detect identical frames by hashing files

import hashlib
from collections import defaultdict

def hash_file(path: str) -> str:
    h = hashlib.md5()
    with open(path, 'rb') as f:
        while chunk := f.read(8192):
            h.update(chunk)
    return h.hexdigest()


def find_exact_duplicates(folder_path: str) -> dict:
    """Return dict of hash -> [filepaths] for duplicates.

    Only identical (byte-wise) duplicates are detected. For near-duplicates
    or static scenes, consider perceptual hashing or frame-diff methods.
    """
    hashes = defaultdict(list)
    for root, dirs, files in os.walk(folder_path):
        for fname in files:
            if not fname.lower().endswith(('.png', '.jpg', '.jpeg')):
                continue
            fp = os.path.join(root, fname)
            try:
                h = hash_file(fp)
                hashes[h].append(fp)
            except Exception:
                pass
    return {h: ps for h, ps in hashes.items() if len(ps) > 1}

# Example usage:
# dups = find_exact_duplicates(dataset_path)
# for h, paths in dups.items(): print(h, len(paths))


In [38]:
# Cell 10: Identify and Remove Corrupted Images
# PURPOSE: Find and remove PNG files that are 3KB or smaller (corrupted/unusable)
# THRESHOLD: 3KB (3072 bytes) - files below this are considered corrupted

# Ensure `data_path` is defined. Try common kagglehub cache locations.
try:
    _ = data_path  # noqa: F821
except NameError:
    dataset_name = 'hcmc-aqi-traffic-camera-dataset'
    possible_roots = [
        os.path.expanduser('~/.cache/kagglehub'),
        '/Users/ianmorris/Documents/Projects/data/kagglehub'
    ]
    found = None
    for root in possible_roots:
        if os.path.exists(root):
            for walk_root, dirs, files in os.walk(root):
                # look for a 'datasets' subfolder containing our dataset
                if 'datasets' in dirs and dataset_name in walk_root:
                    candidate = os.path.join(walk_root, 'datasets')
                    if os.path.exists(candidate):
                        found = candidate
                        break
            if found:
                break
    if found:
        data_path = found
        print(f"Auto-detected data_path: {data_path}")
    else:
        raise NameError("data_path not defined. Set `data_path` to your dataset folder or run the download cell.")

# Find all PNG files and check file sizes
corrupted_files = []
total_size_removed = 0
corrupted_count = 0

# Walk through all image folders
for folder_name in os.listdir(data_path):
    folder_path = os.path.join(data_path, folder_name)
    
    if os.path.isdir(folder_path):
        # Check each PNG file in the folder
        for filename in os.listdir(folder_path):
            if filename.endswith('.png'):
                filepath = os.path.join(folder_path, filename)
                file_size = os.path.getsize(filepath)
                
                # Check if file is corrupted (3KB or smaller)
                if file_size <= 3072:  # 3KB in bytes
                    corrupted_files.append({
                        'path': filepath,
                        'folder': folder_name,
                        'filename': filename,
                        'size': file_size
                    })
                    corrupted_count += 1
                    total_size_removed += file_size

# Display results
print(f"Corrupted Images Found: {corrupted_count}")
print(f"Total Size of Corrupted Files: {total_size_removed / 1024:.2f} KB\n")

if corrupted_files:
    print("Corrupted files by folder:")
    folder_corruption = {}
    for file_info in corrupted_files:
        folder = file_info['folder']
        if folder not in folder_corruption:
            folder_corruption[folder] = 0
        folder_corruption[folder] += 1
    
    for folder, count in sorted(folder_corruption.items()):
        print(f"  {folder}: {count} corrupted files")
    
    # Ask if you want to remove them (optional)
    print("\nTo remove corrupted files, uncomment and run the code below:")
    print("# for file_info in corrupted_files:")
    print("#     os.remove(file_info['path'])")
    print("#     print(f\\\"Removed: {file_info['folder']}/{file_info['filename']}\\\")")
else:
    print("No corrupted files found!")

Corrupted Images Found: 0
Total Size of Corrupted Files: 0.00 KB

No corrupted files found!


## Search for Frozen Frames

In [39]:
# Cell 7: Visualization skeleton
# Purpose: helper to display a grid of images given file paths

import matplotlib.pyplot as plt


def show_image_grid(paths: list, cols: int = 4, thumb_size: tuple[int, int] = (224, 224)):
    """Display images in a grid.

    TODO: implement resizing and plotting; this is a convenience for
    quickly verifying retrieved/top-k examples.
    """
    # TODO: implement plotting
    raise NotImplementedError("TODO: implement show_image_grid")


In [40]:

# Scan each folder for exact duplicates (frozen frames)
all_duplicates = {}

for folder_name in os.listdir(data_path):
    folder_path = os.path.join(data_path, folder_name)
    
    if os.path.isdir(folder_path):
        duplicate_files = find_exact_duplicates(folder_path)
        if duplicate_files:
            all_duplicates[folder_name] = duplicate_files

# Print the results
if all_duplicates:
    print("Exact duplicate files (frozen frames) found:\n")
    for folder_name, duplicate_files in all_duplicates.items():
        print(f"📁 {folder_name}:")
        for file_hash, file_paths in duplicate_files.items():
            print(f"  Hash: {file_hash} ({len(file_paths)} copies)")
            for path in file_paths:
                print(f"    - {os.path.basename(path)}")
        print()
else:
    print("✓ No exact duplicate files (frozen frames) found in any folder!")

Exact duplicate files (frozen frames) found:

📁 NVL_HTP2:
  Hash: 637260e06c707ddb152e19b1df3e0e2f (2 copies)
    - 20-10-2025-11-03-44.png
    - 20-10-2025-10-58-34.png
  Hash: 767d913ef1fddcbecbc013891a2db774 (3 copies)
    - 20-10-2025-12-00-33.png
    - 20-10-2025-11-55-23.png
    - 20-10-2025-11-50-15.png
  Hash: 5fee10b69c14bde6742bd5d10b61cefd (3 copies)
    - 19-10-2025-12-48-02.png
    - 19-10-2025-12-42-50.png
    - 19-10-2025-12-53-12.png
  Hash: 34ec44943f254cdf27b846f3a83bf6bf (2 copies)
    - 19-10-2025-12-11-36.png
    - 19-10-2025-12-16-48.png

📁 LXO_CTL:
  Hash: 16afa6ace8e665561d11ef9d884aa1a7 (3 copies)
    - 20-10-2025-11-45-04.png
    - 20-10-2025-11-39-56.png
    - 20-10-2025-11-50-14.png
  Hash: 1217c1d2ccdf262a10f9bc9d5ba7569b (3 copies)
    - 19-10-2025-13-24-20.png
    - 19-10-2025-13-19-06.png
    - 19-10-2025-13-13-53.png
  Hash: 8e98be4198169cf1d16a6c09a12802b0 (2 copies)
    - 19-10-2025-00-46-14.png
    - 19-10-2025-00-41-05.png
  Hash: 89439078eff7195600